# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
path = "../WeatherPy/output_data/cities.csv"

cityDF = pd.read_csv(path)
cityDF.head()

#citiesDF = citiesDF.iloc[0:50] 
#cityDF.head()
print(len(cityDF))

598


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

cityDF = cityDF.dropna()

# Store 'Lat' and 'Lng' into  locations 
locations = cityDF[["Lat", "Lng"]].astype(float)

Humidity = cityDF["Humidity"].astype(float)

MaxHumidity = cityDF["Humidity"].max()


In [5]:
# Create a Heatmap layer

fig = gmaps.figure(zoom_level =2, center = (14.7,17.5))

heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=MaxHumidity,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
print(len(locations))

550


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:

print("Original cityDF rows:"+str(len(cityDF)))
IdealWeather  = cityDF[cityDF.Humidity <= 55]
IdealWeather  = IdealWeather[IdealWeather["Max Temp"] >= 55 ]
IdealWeather  = IdealWeather[IdealWeather["Max Temp"] < 75]
IdealWeather  = IdealWeather[IdealWeather["Cloudiness"] < 50]
IdealWeather = IdealWeather.dropna()
IdealWeather = IdealWeather.reset_index()
print("IdealWeather rows:"+str(len(IdealWeather)))

IdealWeather.head()

Original cityDF rows:550
IdealWeather rows:21


,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,37,pokaran,26.9167,71.9167,70.92,21.0,0.0,7.90,IN,1.613321e+09
1,126,ginir,7.1333,40.7000,64.15,37.0,10.0,4.79,ET,1.613321e+09
2,138,rio gallegos,-51.6226,-69.2181,57.20,30.0,40.0,23.02,AR,1.613321e+09
3,187,bafq,31.6128,55.4107,60.75,28.0,0.0,3.69,IR,1.613321e+09
4,208,comodoro rivadavia,-45.8667,-67.5000,64.40,42.0,40.0,10.36,AR,1.613321e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = IdealWeather
hotel_df["Hotel Name"]=""


In [9]:
# find the closest restaurant of each type to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    Lat = row['Lat']
    Lng = row['Lng']
    

    params = {
    "radius": "5000",
    "type": "hotel",
    "key": g_key,}
    
    params['location'] = f"{Lat},{Lng}"
    
    # assemble url and make API request
    print(f"Retrieving Results for Index {index}")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Nearby hotel: {results[1]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[1]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 0
Nearby hotel: Khiwaj Mata solanki ki kuldeevi.
------------
Retrieving Results for Index 1
Nearby hotel: Sura Dare Village.
------------
Retrieving Results for Index 2
Nearby hotel: Hotel Santa Cruz.
------------
Retrieving Results for Index 3
Nearby hotel: Hazrat-e- Vali-ye-asr Mosque.
------------
Retrieving Results for Index 4
Nearby hotel: Hotel del Mar.
------------
Retrieving Results for Index 5
Nearby hotel: Pazo de Verdes.
------------
Retrieving Results for Index 6
Nearby hotel: Royal Exchange Hotel.
------------
Retrieving Results for Index 7
Nearby hotel: Mateshwari Mobiles.
------------
Retrieving Results for Index 8
Nearby hotel: Ahmad Al Tobol.
------------
Retrieving Results for Index 9
Nearby hotel: Hotel Arena Gruesa.
------------
Retrieving Results for Index 10
Nearby hotel: Rajasthan Marudhara Grameen Bank.
------------
Retrieving Results for Index 11
Nearby hotel: Trade and Development Bank Branch gallo.
------------
Retrieving Results

In [10]:
hotel_df.head(100)

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,37,pokaran,26.9167,71.9167,70.92,21.0,0.0,7.90,IN,1.613321e+09,Khiwaj Mata solanki ki kuldeevi
1,126,ginir,7.1333,40.7000,64.15,37.0,10.0,4.79,ET,1.613321e+09,Sura Dare Village
2,138,rio gallegos,-51.6226,-69.2181,57.20,30.0,40.0,23.02,AR,1.613321e+09,Hotel Santa Cruz
3,187,bafq,31.6128,55.4107,60.75,28.0,0.0,3.69,IR,1.613321e+09,Hazrat-e- Vali-ye-asr Mosque
4,208,comodoro rivadavia,-45.8667,-67.5000,64.40,42.0,40.0,10.36,AR,1.613321e+09,Hotel del Mar
5,226,bereda,43.2686,-7.5406,63.00,55.0,0.0,17.27,ES,1.613321e+09,Pazo de Verdes
6,234,broken hill,-31.9500,141.4333,66.20,27.0,0.0,6.91,AU,1.613321e+09,Royal Exchange Hotel
7,286,uniara,26.1500,75.2167,64.65,29.0,4.0,3.49,IN,1.613321e+09,Mateshwari Mobiles
8,312,yabrud,33.9674,36.6574,60.80,36.0,0.0,5.75,SY,1.613321e+09,Ahmad Al Tobol
9,320,ancud,-41.8697,-73.8203,60.80,48.0,0.0,12.66,CL,1.613321e+09,Hotel Arena Gruesa


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotellocations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
fig = gmaps.figure(zoom_level =2, center = (14.7,17.5))

heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=MaxHumidity,
                                 point_radius = 3)
marker_layer = gmaps.marker_layer(
        hotellocations,
        hover_text=hotel_info
    )

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))